In [16]:
from itertools import combinations
from collections import Counter
from evaluateHand import evaluate_hand

In [15]:
# Define card rank and suit mappings
RANKS = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
SUITS = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
DECK = [(rank, suit) for rank in RANKS for suit in SUITS]

In [32]:
"""
Evalutes prob of each hand of the next possible hand given the current community hand and the player's hand
"""


def hand_type_probability(current_community_cards, hand_cards):
    all_known_cards = set(current_community_cards + hand_cards)
    remaining_deck = [card for card in DECK if card not in all_known_cards]
    
    hand_types = {
        9: 'Royal Flush',
        8: 'Straight Flush',
        7: 'Four of a Kind',
        6: 'Full House',
        5: 'Flush',
        4: 'Straight',
        3: 'Three of a Kind',
        2: 'Two Pair',
        1: 'One Pair',
        0: 'High Card'
    }
    
    hand_counts = {hand: 0 for hand in hand_types.values()}

    total_possible_hands = 0
    num_known_cards = len(current_community_cards) + len(hand_cards)

    # Calculate probabilities based on the number of cards
    if num_known_cards == 5:
        num_needed = 1
        possible_next_cards = remaining_deck
    elif num_known_cards == 6:
        num_needed = 1
        possible_next_cards = remaining_deck
    elif num_known_cards == 7:
        num_needed = 0
        possible_next_cards = []  # No more cards to draw
    else:
        raise ValueError("Invalid number of known cards")

    if num_needed > 0:
        for next_card in possible_next_cards:
            # Simulate the hand with the next card
            complete_hand = current_community_cards + hand_cards + [next_card]
            # Generate all 5-card combinations from the complete hand
            possible_hands = list(combinations(complete_hand, 5))
            for hand in possible_hands:
                hand_type, _ = evaluate_hand(hand)
                hand_name = hand_types[hand_type[1]]
                hand_counts[hand_name] += 1
            total_possible_hands += len(possible_hands)
    else:
        # No more cards to draw, just evaluate the current hand
        complete_hand = current_community_cards + hand_cards
        hand_type, _ = evaluate_hand(complete_hand)
        hand_name = hand_types[hand_type[1]]
        hand_counts[hand_name] = 1
        total_possible_hands = 1

    probabilities = {hand: count / total_possible_hands for hand, count in hand_counts.items()}
    
    return probabilities

# Example usage
community_cards = [('A', 'Hearts'), ('K', 'Hearts'), ('Q', 'Spades')]  # Example community cards
hand_cards = [('J', 'Hearts'), ('10', 'Hearts')]  # Example hand cards

# First round: 5 cards, predicting probabilities with 6th card
print("Probabilities with next card:")
probabilities = hand_type_probability(community_cards, hand_cards)
for hand_type, prob in probabilities.items():
    print(f"{hand_type}: {prob:.4f}")


Probabilities with next card:
Royal Flush: 0.0035
Straight Flush: 0.0000
Four of a Kind: 0.0000
Full House: 0.0000
Flush: 0.0284
Straight: 0.2305
Three of a Kind: 0.0000
Two Pair: 0.0000
One Pair: 0.2128
High Card: 0.5248


Now define a single prob factor combining all the hands combinations probabilty and their rank (1 for Royal Flush) and use this to determine the bots move

#### Note: This is only an example thus the prob function is inefficient and Teams are encourage to come up with their own functions

Now make use of the produced probability to decide the outcome
The three approaches that can be made (with increasing complexity)
1. Define a prob-risk limit till when u can fold,check,raise
2. Tweak your prob-risk factor when the opponent raises/folds
3. Use Algorithms to account for strategy and patterns of the opponent (Example: Counterfactual Regret Minimization - CFT)

Next Steps include 
- Keeping in memory of the community cards and taking input each round (of 3)
- Defining a variable for your pool (points) that are spent each round
- Defining what the bot should do each round based on your prob-risk factor

Your inputs for each round should (only) include
- The new community card delt
- what the opponents have played (call/raise/fold)

